## Applying LASSO: Computational Packages and Some Practical Issues

#### Rodolfo Lourenzutti
#### 2023-05-30

# Lecture Plan and Objectives

In this part 2 of the lecture, we will:
- review the video material;
- learn how to fit a LASSO model using `glmnet` and `cv.glmnet`;
    - we will code this together! 
- discuss some caveats of LASSO in inference and categorical variables;

# Review Questions

**True or False:**

We must standardize the predictors before applying LASSO. 

Why? 

$$
LASSO(\beta) = \sum_{i=1}^n \left(y_i - \beta_0 - \beta_1X_{i1} - \ldots - \beta_p X_{ip}\right)^2 + {\color{red}{\lambda\sum_{i=1}^p \left|\beta_i\right|}}
$$

**True or False:**

The penalty term compromises LASSO's ability to fit the intercept term properly. For this reason, we should always center the response.

**True or False:**

When we increase the $\lambda$ value of the penalty term, the absolute value of each coefficient will be reduced by LASSO. 

- Imagine you have a restriction that $|\beta_1|$ and $|\beta_2|$ must be less than 1. 
  - You could have $|\beta_1| = 0.6$ and $|\beta_2| = 0.4$.
  
- Now, if we reduce our budget to 0.7. Possible results are:
   - $|\beta_1| = 0.7$ and $|\beta_2| = 0$.
   - $|\beta_1| = 0$ and $|\beta_2| = 0.7$.

- The main idea is that the set of variables selected in a model with 4 variables might have different variables than in a model with 3 variables, or with 2 variables, and so on. 

**True or False:**

When we increase the $\lambda$ value of the penalty term, the sum of the absolute values of the coefficients will be reduced by LASSO. 

**True or False:**

LASSO's solution improves the Residual Sum of Squares in comparison to OLS, which is one of the reasons LASSO is superior to OLS. 

In [ ]:
# Loading Packages

library(tidyverse)
library(gridExtra)
library(tidymodels)
library(broom)
library(leaps)
library(Brq)
library(glmnet)
library(knitr)

## Review the LASSO method

In LASSO, change the OLS objective function by adding a penalty term to it:

$$
LASSO(\beta) = \sum_{i=1}^n \left(y_i - \beta_0 - \beta_1X_{i1} - \ldots - \beta_p X_{ip}\right)^2 + {\color{red}{\lambda\sum_{i=1}^p \left|\beta_i\right|}}
$$

- LASSO regularizes **and** selects features simultaneously;

- The penalty term keeps the coefficients in check;
    1. Large values of $\lambda$ will...?
    2. Small values of $\lambda$ will...?




- LASSO has no closed-form solution. We need to use algorithms; 

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)


lasso_large_lambda <- ggplot(tibble(x1 = -0.5,  x2 = 0, x3 = 0.5, x4 = 0,
              y1 = 0, y2 = 0.5, y3 = 0, y4 = -0.5)) + 
  geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2)) +
  geom_segment(aes(x = x2, y = y2, xend = x3, yend = y3)) +
  geom_segment(aes(x = x3, y = y3, xend = x4, yend = y4)) +
  geom_segment(aes(x = x1, y = y1, xend = x4, yend = y4)) +
  labs(x = 'β1', y = 'β2', title = 'Large Lambda') +
  xlim(-1, 1) +
  ylim(-1, 1) +
  geom_point(aes(x,y), color = "darkgreen", data = tibble(x=c(0.78), y=c(0.25)), size = 5) + 
  annotate("text", x = 0.78-0.025, y = 0.18, label = "OLS solution", size = 8) + 
  theme(text = element_text(size = 30))


lasso_small_lambda <- ggplot(tibble(x1 = -2,  x2 = 0, x3 = 2, x4 = 0,
              y1 = 0, y2 = 2, y3 = 0, y4 = -2)) + 
  geom_segment(aes(x = x1, y = y1, xend = x2, yend = y2)) +
  geom_segment(aes(x = x2, y = y2, xend = x3, yend = y3)) +
  geom_segment(aes(x = x3, y = y3, xend = x4, yend = y4)) +
  geom_segment(aes(x = x1, y = y1, xend = x4, yend = y4)) +
  labs(x = 'β1', y = 'β2', title = 'Small Lambda') +
  xlim(-3, 3) +
  ylim(-3, 3) +
  geom_point(aes(x,y), color = "darkgreen", data = tibble(x=c(0.78), y=c(0.25)), size = 5) + 
  annotate("text", x = 0.78-0.02, y = 0.025 + 0.06, label = "OLS solution", size = 8) + 
  theme(text = element_text(size = 30))

In [ ]:
grid.arrange(lasso_large_lambda, lasso_small_lambda, ncol=2)

## Fitting LASSO 

### Prostate Cancer Data Problem

- Do you want to learn how to fit LASSO to a dataset? 

![](https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExNDYxMzliMDc1NmRlMzVjMGFlZWVkZGIyZDI0MDA3MzRlZGU5M2M4NCZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/fypu26wRfQZVgu0Prx/giphy.gif)

### Let's start by loading the data

In [ ]:
# Loading the data
data("Prostate")

cancer_prostate <- 
    tibble(Prostate) 

# Printing the first 10 rows
head(cancer_prostate, 10)

### Data Dictionary
- **lcavol**: log(cancer volume) 
- **lweight**: log(prostate weight)
- **age**: age of the patient
- **lbph**: log(amount of benign prostatic hyperplasia)
- **svi**: Seminal vesicle invasion. (True or False)
- **lcp**: log(capsular penetration)
- **gleason**: Gleason score. The Gleason score measures how abnormal the tissue looks. The lower the score is, the more the cells look like regular prostate tissue. You can learn more about the [Gleason Score here](https://www.pcf.org/about-prostate-cancer/diagnosis-staging-prostate-cancer/gleason-score-isup-grade/)
- **pgg45**: Percentage Gleason scores 4 or 5. Scores 4 and 5 are considered very high; the cells barely look like normal prostate tissue.
- **lpsa**: log(prostate-specific antigen)

- Unfortunately, the units for the variables are not readily available and are unknown. 

- However, remember that the units are fundamental to interpreting the model. 

- So always make sure you know what the units are (even though I'm not sure what the units are in this case &#x1F609;). 


## Step 1: Data splitting

Even though we will be fitting the LASSO method and not comparing it with other models (such as., k-nn regression), there's still plenty we still need to learn. For example, we still need to figure out the following:

1. which covariates are relevant? 
2. how much regularization should we use?

We will use the data to make these decisions. But once we've selected the model, we need to be able to assess our model. We must use different data to do this. 

### Exercise 1

So let's start by splitting the dataset into two sets: (1) train set, with 70% of the rows; and (2) test set, with the remaining 30%. 

_Save the train set in an object called `cancer_train` and test set in `cancer_test`._



In [ ]:
set.seed(20230530) # Do not change this

cancer_prostate_split <- initial_split(..., prop = ...)

cancer_train <- training(...)
cancer_test <- testing(...) 

### Exercise 2 

Let's take a look at some summary quantities of our data. First, remove the `svi` variable (we don't want to calculate the summary of a binary variable). 

In [ ]:
... |> 
    ...(...) |>
    summary()

### Step 2: Data Standardization

- <span style='color: red'>DO NOT FORGET:</span> we need to standardize the data before applying LASSO!

- Luckily for us, `glmnet` automatically standardizes the data for us! 


## Step 3: Calling  the `glmnet` function

- `glmnet` is much more flexible than we need here. 

- So, let's focus only on the arguments that matter to us;

- You can learn more with [`glmnet`'s vignette](https://glmnet.stanford.edu/articles/glmnet.html) or calling help `?glmnet`



### Exercise 3

To train a regression using LASSO, we can use the package `glmnet` in R. A few points to keep in mind:

- `glmnet` actually uses a penalty function that generalizes LASSO. So, to reduce that generalization to LASSO, we set alpha = 1, which is the default. 


- we can define a grid of values to evaluate by providing `lambda`, or we can just say how many lambdas we want to try out, and the function will define the values on its own. 

Now, fill in the code below to run LASSO with our `cancer_train` dataset. We want to try out 200 values for `lambda`. Store the model in an object named `lasso_cancer_fitted`.

In [ ]:
set.seed(20230530)

# Fill in the code

lasso_cancer_fitted <- # The variable to store the fitted model
    glmnet(
        x = ... |> select(-...),  # First argument is the covariates (note that we need at least two columns)
        y = cancer_train$...,  # Second argument is the response
        alpha = 1, # This is the default and is equivalent to LASSO 
        nlambda = ..., # the number of lambda values to try (default is 100)
        #lambda  = ..., # alternatively, the values of lambda you want try
        standardize = TRUE, # Default value is TRUE
    )




## Step 4: Extracting information from fitted model

- The fitted model is a complex R object. 

- We will use some functions to extract the main parts of our model

### Exercise 4: the solution Path

If we call the `plot` function directly on the fitted model and it will show the value of the coefficients for different values of lambda. We need to specify `xvar = 'lambda'`.

In [ ]:
...(..., xvar = 'lambda')

### Exercise 5: grabbing the coefficients

To see what the coefficients are, we call the `coef` function. Naturally, the values of coefficients depend on lambda. We can specify one or more values of lambda using the `s` argument. 
- If you don't specify an `s` value, coefficients for all fitted $\lambda$ will be returned;
- If you pick a $\lambda$ not fitted by the model, it returns an "approximation" of the coefficients via interpolation; usually, this approximation is quite accurate. 

Obtain the coefficients for the model with $\lambda=0.4$ and $\lambda=0.1$

In [ ]:
coef(..., s = c(...)) # here lambda is called `s` (as in "size" of the model, but lower s --> larger models)

### Example 6: 

You can specify `exact = TRUE` to fit the model for a given $\lambda$ and obtain the exact coefficients.
  - In this case, we need to provide $x$ and $y$ again;

In [ ]:
coef(lasso_cancer_fitted, 
     s = 0.4, 
     exact = TRUE, 
     x = cancer_train |> select(-lpsa),
     y = cancer_train$lpsa)

## Step 5: making predictions

### Exercise 6

For prediction, the flow is very similar. We call the `predict` function and specify one or more lambda values we want in the `s` argument, but we also need to provide the `newx` matrix containing the data to be used for prediction. 

Unfortunately, `glmnet` doesn't play well with data frames and requires a matrix for `newx`.

Use `cancer_train` data to obtain the predictions for the model `lasso_cancer_fitted` using  $\lambda=0.4$ and $\lambda=0.1$.

In [ ]:
     
predict(lasso_cancer_fitted, # Fitted model
    newx = ... |> select(-...) |> as.matrix(), # A matrix (yes, tibble don't work) containing the data for prediction,
    s = c(..., ...) # one or more values of lambda
   ) 

# Tuning: the process of selecting  $\lambda$: 

- We saw that `glmnet` fits LASSO for many different values of $\lambda$. Which $\lambda$ should we use? 

- This is called hyper-parameter tuning, and we usually do this using cross-validation. 

- We decide on a metric, such as MSE, and pick $\lambda$ with the best performance in the cross-validation set. 

### Exercise 7

Luckily for us, we don't need to do this ourselves. Instead, we can call `cv.glmnet`, which will do this entire process for us. The function is very similar to `glmnet`. Fill in the code below to run `cv.glmnet`. 

_Save the fitted model in an object called `cancer_cv_lasso`._

In [ ]:
cancer_cv_lasso <- 
    cv.glmnet(
        x = ... |> ... |> as.matrix(), # Covariates
        y = ..., # Response
        lambda = NULL, # A sequence of lambdas. Default lets the function choose its own sequence. 
        nfolds = 10, # number of folds to be used in the cross validation
        standardize = TRUE
    )

print(cancer_cv_lasso)

## `plot` functions for `cv.glmnet`

The `coef` and `predict` functions work exactly the same as for the `glmnet`. But the plot function is a little different. Let's check! 

### Exercise 8

Call the `plot` function directly on the fitted `cv.glmnet` model. 

In [ ]:
plot(...)

- The plot shows the estimated MSE for different value of lambdas; 

- The numbers at the top show how many variables are included in the model for each lambda; 

- The error bars represent the variation of MSE over the different folds of the cross-validation;

- First vertical dotted line (on the left), the best CV $\lambda$ value; 

- Second vertical dotted line (on the right), the most regularized model (highest $\lambda$) that the CV performance is 1 std. dev. from the best one. 

## `coef`, and `predict` functions for `cv.glmnet`

- They work pretty much the same way;

- Except now we can use two string for `s` argument:
    - "lambda.1se"
    - "lambda.min"

In [ ]:
coef(cancer_cv_lasso, 
    s = 'lambda.1se') # two new possible values:  "lambda.min" or "lambda.1se" (default)

In [ ]:
coef(cancer_cv_lasso, 
    s = 'lambda.min')

In [ ]:
predict(
    cancer_cv_lasso, 
    newx = cancer_train |> select(-lpsa) |> as.matrix(),
    s = 'lambda.1se') 

# A couple of "caveats"


## LASSO and categorical variable

- When we fit categorical data, we use dummy variables (one-hot-encoding); 


- What happens if LASSO removes one of the dummy variables but leaves the others? Does it make sense? 


- There are some LASSO extensions, such as Group LASSO, that deal with this. 

### LASSO Inference

- The LASSO method gives biased estimates for the parameters; 
    - Biased estimates are not the end of the world, but can make inference trickier; 
    
- Confidence intervals are not immediately available for LASSO
   - There are a few proposals for confidence intervals available for coefficients and hypothesis testing for LASSO models; 

- The trickier part is that inference for LASSO triggers the post-selection inference problem;
  - I have prepared an activity to walk you through this problem and illustrate the bias in the LASSO estimate. 
  
  
- An approach to deal with this is to use data split. 
    - In the first split, we run LASSO to select the variables; 
    - In the second split, we fit OLS and use that inference; 